In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import time
from selenium import webdriver
import os
import random
import pyprind

In [2]:
my_headers = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ' + "(KHTML, like Gecko) Chrome/61.0.3163.100Safari/537.36"
}

## Procedure

+ Download Search Pages
+ Parse all individual links (relavant to intership post)
+ Download All intership post pages
+ Parse and Extract all content of intership pages

### Downloading Search Pages

In [3]:
def scroll_down(driver):
    driver.set_window_size(1440, 875)
    driver.execute_script("window.scrollTo(0,397)")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,24695)")

def get_parameterized_url(start_after):
    return f'https://internshala.com/internships/computer%20science-internship/page-{start_after}'


def download_page(page_src,name):
    os.environ['MOZ_HEADLESS'] = '1'
    driver = webdriver.Firefox(executable_path='/Users/admin/Projects/Msc/Data Mining/Scrapping/geckodriver')
    driver.get(page_src)
    scroll_down(driver)
    with open(f"../raw/{name}.html", "w") as f:
        f.write(driver.page_source)
    driver.close()
    
def requests_download_page(URL, filename):
    res = requests.get(URL, headers=my_headers)
    with open(f"../raw/{filename}.html", "w") as f:
        f.write(res.text)

In [4]:
TOTAL_PAGES = 50

bar = pyprind.ProgBar(TOTAL_PAGES)
for i in range(1, TOTAL_PAGES + 1):
    requests_download_page(get_parameterized_url(i), f"internshala-june/search-pages/search-page-{i}")
    time.sleep(1)
    if(i % 10 == 0):
        time.sleep(random.randint(2,4))
    bar.update()
print(bar)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:23


Title: 
  Started: 07/01/2021 09:27:53
  Finished: 07/01/2021 09:30:17
  Total time elapsed: 00:02:23


#### Analyse the strucutre and logic to extract Internship post links (href in View Details button)

In [19]:
path = '../raw/internshala/search-pages/search-page-1.html'
bs = BeautifulSoup(open(path))
post_links = bs.find_all(class_='view_detail_button')

In [21]:
post_links[0]

<a class="view_detail_button" href="/internship/detail/information-technology-work-from-home-job-internship-at-child-rights-you1622116164">
<div>View details</div>
<i class="ic-16-chevron-right"></i>
</a>

In [25]:
post_links[0].attrs

{'class': ['view_detail_button'],
 'href': '/internship/detail/information-technology-work-from-home-job-internship-at-child-rights-you1622116164'}

### Parsing all individual links

In [5]:
ALL_INTERNSHIP_POST_LINKS = []

In [6]:
search_pages_filenames = os.listdir('../raw/internshala-june/search-pages')

bar = pyprind.ProgBar(len(search_pages_filenames))
for page in search_pages_filenames:
    with open(f'../raw/internshala-june/search-pages/{page}', 'r') as file:
        soup = BeautifulSoup(file)
        all_links = soup.find_all(class_='view_detail_button')
        for link in all_links:
            try:
                attributes = link.attrs
                ALL_INTERNSHIP_POST_LINKS.append(attributes['href'])
            except:
                print('Error finding attributes')
    bar.update()
bar

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Title: 
  Started: 07/01/2021 09:31:37
  Finished: 07/01/2021 09:31:45
  Total time elapsed: 00:00:07

In [7]:
print(f'Numebr of Links = {len(ALL_INTERNSHIP_POST_LINKS)}')
print(f"Example Link = {ALL_INTERNSHIP_POST_LINKS[0]}")

Numebr of Links = 2000
Example Link = /internship/detail/reactjs-development-work-from-home-job-internship-at-iit-bombay1624886518


In [8]:
# Complete the URL by adding TLDs - i.e prepend TLD to page path
TLD = 'https://internshala.com'
for i in range(len(ALL_INTERNSHIP_POST_LINKS)):
    ALL_INTERNSHIP_POST_LINKS[i] = TLD + ALL_INTERNSHIP_POST_LINKS[i]
ALL_INTERNSHIP_POST_LINKS[0]

'https://internshala.com/internship/detail/reactjs-development-work-from-home-job-internship-at-iit-bombay1624886518'

In [9]:
# Save the links to file
with open('internship-post-links-june.txt','w') as file:
    file.write('\n'.join(ALL_INTERNSHIP_POST_LINKS))

In [10]:
ALL_INTERNSHIP_POST_LINKS[0]

'https://internshala.com/internship/detail/reactjs-development-work-from-home-job-internship-at-iit-bombay1624886518'

### Download All internship post pages

In [17]:
total_length = len(ALL_INTERNSHIP_POST_LINKS)
bar = pyprind.ProgBar(total_length)
for (idx, link) in enumerate(ALL_INTERNSHIP_POST_LINKS[:100]):
    requests_download_page(link, f"internshala-june/internship-pages/page-{idx+1}")
    time.sleep(0.5)
#     if(idx % 10 == 0):
#         time.sleep(random.randint(0,2))
    bar.update()
print(bar)

0% [#                             ] 100% | ETA: 01:06:09

Title: 
  Started: 07/01/2021 10:54:25
  Finished: 07/01/2021 10:57:37
  Total time elapsed: 00:00:00


In [16]:
[x for x in enumerate(ALL_INTERNSHIP_POST_LINKS[1214:])]

[(0,
  'https://internshala.com/internship/detail/web-development-work-from-home-job-internship-at-reev1624604978'),
 (1,
  'https://internshala.com/internship/detail/website-graphics-designing-work-from-home-job-internship-at-hoopa-baby-junction1624548788'),
 (2,
  'https://internshala.com/internship/detail/java-development-internship-in-multiple-locations-at-feednet-solutions-private-limited1624605073'),
 (3,
  'https://internshala.com/internship/detail/web-development-internship-in-delhi-bangalore-at-mobishaala1624604869'),
 (4,
  'https://internshala.com/internship/detail/mobile-app-development-internship-in-delhi-at-mobishaala1624604821'),
 (5,
  'https://internshala.com/internship/detail/web-development-work-from-home-job-internship-at-formula-grey1624601816'),
 (6,
  'https://internshala.com/internship/detail/mobile-app-development-internship-in-delhi-noida-at-one-87-global1624555677'),
 (7,
  'https://internshala.com/internship/detail/web-development-work-from-home-job-internsh